In [27]:
import pandas as pd
import numpy as np
import json
import calculations
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from scipy.stats import iqr, skew, kurtosis

In [28]:
# Process data
   
stimuli = []
scale_type = {
    "gesture_pitch_number": 100,
    "gesture_pitch_greyscale": 49,
    "gesture_roll_number": 100,
    "gesture_roll_greyscale": 49,
    "slider_number": 1,
    "slider_greyscale": 1,
}

with open('experiment_data/experiment_data_14.json', 'r') as file:
    data = json.load(file)

    if data:
        for experiment in data["completedExperiments"]:
            experiment_type = experiment.get("experimentType")
            for stimulus in experiment["successfulStimuli"]:
                sensor_reading = stimulus.get("sensorReading", {})
                quaternion = (
                    sensor_reading.get("x"),
                    sensor_reading.get("y"),
                    sensor_reading.get("z"),
                    sensor_reading.get("w"),
                )
                _, roll, pitch = calculations.quaternion_to_euler(*quaternion)
                
                sensor_reading["roll"] = roll
                sensor_reading["pitch"] = pitch

                pitch_scaled = calculations.scale_pitch(pitch, scale_type[experiment_type])
                roll_scaled = calculations.scale_roll(roll, scale_type[experiment_type])
                truth_value = stimulus.get("truth")
                slider_value = stimulus.get("value")
                
                pitch_difference = abs(truth_value - pitch_scaled)
                roll_difference = abs(truth_value - roll_scaled)

                stimulus["pitch"] = pitch
                stimulus["pitch_truth"] = pitch_scaled
                stimulus["pitch_truth_diff"] = pitch_difference
            
                stimulus["roll"] = roll
                stimulus["roll_truth"] = roll_scaled
                stimulus["roll_truth_diff"] = roll_difference
                
                stimulus["slider_diff"] = abs(truth_value - slider_value)
                
                if truth_value == 0:
                    stimulus["pitch_truth_diff_percentage"] = (pitch_scaled / scale_type[experiment_type]) * 100
                    stimulus["roll_truth_diff_percentage"] = (roll_scaled / scale_type[experiment_type]) * 100
                else:
                    stimulus["pitch_truth_diff_percentage"] = (pitch_difference / truth_value) * 100
                    stimulus["roll_truth_diff_percentage"] = (roll_difference / truth_value) * 100
                    
                stimulus["experiment_type"] = experiment_type
                stimulus["calibration"] = sensor_reading.get("calibration_status")
                
                stimuli.append(stimulus)

df = pd.DataFrame(data=stimuli)

In [29]:
truth_value_batches_100 = [
    [i for i in range(0, 10)],
    [i for i in range(10, 20)],
    [i for i in range(20, 30)],
    [i for i in range(30, 40)],
    [i for i in range(40, 50)],
    [i for i in range(50, 60)],
    [i for i in range(60, 70)],
    [i for i in range(70, 80)],
    [i for i in range(80, 90)],
    [i for i in range(90, 101)]
]

truth_value_batches_49 = [
    [i for i in range(0, 10)],
    [i for i in range(10, 20)],
    [i for i in range(20, 30)],
    [i for i in range(30, 40)],
    [i for i in range(40, 50)]
]

experiment_truth_type = {
    "gesture_pitch_number": {
        "truth": "pitch_truth",
        "truth_diff": "pitch_truth_diff",
        "label": "Pitch Truth",
        "input_type": "device"
    },
    "gesture_pitch_greyscale": {
        "truth": "pitch_truth",
        "truth_diff": "pitch_truth_diff",
        "label": "Pitch Truth",
        "input_type": "device"
    },
    "gesture_roll_number": {
        "truth": "roll_truth",
        "truth_diff": "roll_truth_diff",
        "label": "Roll Truth",
        "input_type": "device"
    },
    "gesture_roll_greyscale": {
        "truth": "roll_truth",
        "truth_diff": "roll_truth_diff",
        "label": "Roll Truth",
        "input_type": "device"
    },
    "slider_number": {
        "truth": "value",
        "truth_diff": "slider_diff",
        "label": "Slider Value",
        "input_type": "slider"
    },
    "slider_greyscale": {
        "truth": "value",
        "truth_diff": "slider_diff",
        "label": "Slider Value",
        "input_type": "slider"
    },
}

In [30]:
df[df["experiment_type"] == "gesture_pitch_greyscale"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,calibration
0,"{'w': 0.00860596, 'z': 0.789124, 'stability': ...",device,0.0,20,1,13.462403,7,13,-15.813298,9,11,20.0,65.000000,55.000000,gesture_pitch_greyscale,2
1,"{'duration': 167, 'x': -0.204102, 'stability':...",device,0.0,5,2,21.499965,12,7,-13.396093,7,2,5.0,140.000000,40.000000,gesture_pitch_greyscale,2
2,"{'z': 0.855957, 'w': -0.0631714, 'stability': ...",device,0.0,48,3,-2.685654,1,47,-9.697546,5,43,48.0,97.916667,89.583333,gesture_pitch_greyscale,2
3,"{'stability': 'In motion', 'z': 0.62384, 'cali...",device,0.0,29,4,81.944114,45,16,-2.046509,1,28,29.0,55.172414,96.551724,gesture_pitch_greyscale,2
4,"{'w': 0.171936, 'duration': 190, 'activity': '...",device,0.0,46,5,36.339965,20,26,-13.023467,7,39,46.0,56.521739,84.782609,gesture_pitch_greyscale,2
5,"{'y': -0.395081, 'stability': 'In motion', 'ac...",device,0.0,0,6,89.156890,49,49,-5.296500,3,3,0.0,100.000000,6.122449,gesture_pitch_greyscale,2
6,"{'y': -0.501587, 'timestamp': 1719319643, 'x':...",device,0.0,28,7,-0.283176,0,28,-6.201000,3,25,28.0,100.000000,89.285714,gesture_pitch_greyscale,2
7,"{'calibration_status': 2, 'x': -0.282837, 'y':...",device,0.0,37,8,36.962015,20,17,-13.806352,8,29,37.0,45.945946,78.378378,gesture_pitch_greyscale,2
8,"{'timestamp': 1719319653, 'y': -0.483765, 'dur...",device,0.0,43,9,68.391026,37,6,-9.929701,5,38,43.0,13.953488,88.372093,gesture_pitch_greyscale,2
9,"{'timestamp': 1719319658, 'calibration_status'...",device,0.0,3,10,74.602687,41,38,-6.827826,4,1,3.0,1266.666667,33.333333,gesture_pitch_greyscale,2


In [31]:
df[df["experiment_type"] == "gesture_pitch_number"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,calibration
20,"{'w': 0.180237, 'x': -0.390442, 'calibration_s...",device,0.0,64,1,45.932695,51,13,-25.042959,28,36,64.0,20.312500,56.250000,gesture_pitch_number,2
21,"{'w': 0.224854, 'x': -0.380798, 'calibration_s...",device,0.0,67,2,49.044438,54,13,-21.739210,24,43,67.0,19.402985,64.179104,gesture_pitch_number,2
22,"{'calibration_status': 2, 'duration': 181, 'y'...",device,0.0,91,3,70.892690,79,12,-17.946928,20,71,91.0,13.186813,78.021978,gesture_pitch_number,2
23,"{'x': -0.392883, 'activity_confidence': 91, 'z...",device,0.0,62,4,48.791011,54,8,-23.174989,26,36,62.0,12.903226,58.064516,gesture_pitch_number,2
24,"{'z': 0.732666, 'timestamp': 1719319747, 'acti...",device,0.0,74,5,59.346314,66,8,-24.082532,27,47,74.0,10.810811,63.513514,gesture_pitch_number,2
25,"{'duration': 190, 'activity_confidence': 94, '...",device,0.0,68,6,48.647061,54,14,-29.189574,32,36,68.0,20.588235,52.941176,gesture_pitch_number,2
26,"{'calibration_status': 2, 'y': -0.493164, 'tim...",device,0.0,28,7,27.000324,30,2,-30.461561,34,6,28.0,7.142857,21.428571,gesture_pitch_number,2
27,"{'timestamp': 1719319768, 'activity_confidence...",device,0.0,56,8,45.326728,50,6,-24.762911,28,28,56.0,10.714286,50.000000,gesture_pitch_number,2
28,"{'z': 0.651733, 'calibration_status': 2, 'dura...",device,0.0,57,9,46.729194,52,5,-24.099269,27,30,57.0,8.771930,52.631579,gesture_pitch_number,2
29,"{'duration': 154, 'y': -0.540833, 'x': -0.5241...",device,0.0,93,10,70.042286,78,15,-20.931239,23,70,93.0,16.129032,75.268817,gesture_pitch_number,2


In [32]:
df[df["experiment_type"] == "gesture_roll_greyscale"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,calibration
40,"{'z': 0.689941, 'x': -0.481995, 'y': -0.472534...",device,0.0,44,1,13.376283,7,37,-65.808874,36,8,44.0,84.090909,18.181818,gesture_roll_greyscale,1
41,"{'activity_confidence': 73, 'y': -0.430176, 'c...",device,0.0,23,2,22.423382,12,11,-75.377780,41,18,23.0,47.826087,78.260870,gesture_roll_greyscale,1
42,"{'duration': 191, 'w': -0.209595, 'activity_co...",device,0.0,36,3,2.314338,1,35,-48.042855,26,10,36.0,97.222222,27.777778,gesture_roll_greyscale,1
43,"{'duration': 154, 'activity_confidence': 83, '...",device,0.0,9,4,2.277466,1,8,-51.946909,28,19,9.0,88.888889,211.111111,gesture_roll_greyscale,1
44,"{'duration': 191, 'timestamp': 1719319897, 'w'...",device,0.0,25,5,1.087700,1,24,-28.642069,16,9,25.0,96.000000,36.000000,gesture_roll_greyscale,2
45,"{'calibration_status': 1, 'duration': 154, 'st...",device,0.0,32,6,0.670309,0,32,-39.366854,21,11,32.0,100.000000,34.375000,gesture_roll_greyscale,1
46,"{'y': 0.219299, 'timestamp': 1719319938, 'stab...",device,0.0,49,7,1.383910,1,48,-62.179796,34,15,49.0,97.959184,30.612245,gesture_roll_greyscale,2
47,"{'y': -0.263794, 'z': 0.584167, 'calibration_s...",device,0.0,4,8,179.123936,49,45,-79.725375,43,39,4.0,1125.000000,975.000000,gesture_roll_greyscale,1
48,"{'timestamp': 1719319973, 'calibration_status'...",device,0.0,6,9,-0.864269,0,6,-25.683753,14,8,6.0,100.000000,133.333333,gesture_roll_greyscale,2
49,"{'duration': 208, 'w': -0.141602, 'x': -0.2551...",device,0.0,24,10,-1.246253,1,23,-33.909533,18,6,24.0,95.833333,25.000000,gesture_roll_greyscale,2


In [33]:
df[df["experiment_type"] == "gesture_roll_number"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,calibration
60,"{'duration': 213, 'activity_confidence': 95, '...",device,0.0,75,1,-6.540093,7,68,-64.960044,72,3,75.0,90.666667,4.000000,gesture_roll_number,2
61,"{'activity_confidence': 96, 'w': -0.284912, 's...",device,0.0,81,2,-6.503005,7,74,-69.505106,77,4,81.0,91.358025,4.938272,gesture_roll_number,2
62,"{'z': 0.770508, 'stability': 'Stable', 'y': -0...",device,0.0,70,3,-5.603559,6,64,-59.262947,66,4,70.0,91.428571,5.714286,gesture_roll_number,2
63,"{'activity': 'Still', 'calibration_status': 2,...",device,0.0,26,4,-4.109254,5,21,-36.102319,40,14,26.0,80.769231,53.846154,gesture_roll_number,2
64,"{'calibration_status': 2, 'activity': 'Still',...",device,0.0,20,5,-3.932020,4,16,-33.662836,37,17,20.0,80.000000,85.000000,gesture_roll_number,2
65,"{'calibration_status': 2, 'y': -0.397583, 'tim...",device,0.0,63,6,-6.837365,8,55,-58.008506,64,1,63.0,87.301587,1.587302,gesture_roll_number,2
66,"{'z': 0.73645, 'stability': 'Stable', 'timesta...",device,0.0,85,7,-7.991331,9,76,-67.340333,75,10,85.0,89.411765,11.764706,gesture_roll_number,2
67,"{'activity_confidence': 80, 'activity': 'Still...",device,0.0,100,8,179.464782,100,0,-77.994724,87,13,100.0,0.000000,13.000000,gesture_roll_number,1
68,"{'activity': 'Still', 'stability': 'Stable', '...",device,0.0,96,9,-176.259097,100,4,-86.372440,96,0,96.0,4.166667,0.000000,gesture_roll_number,1
69,"{'calibration_status': 2, 'duration': 231, 'x'...",device,0.0,46,10,-7.100933,8,38,-59.022820,66,20,46.0,82.608696,43.478261,gesture_roll_number,2


In [34]:
df[df["experiment_type"] == "slider_number"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,calibration
100,"{'stability': 'In motion', 'activity_confidenc...",slider,91.702067,89,1,-7.474058,0,89,-36.446336,0,89,2.702067,100.0,100.0,slider_number,2
101,"{'z': 0.847412, 'activity_confidence': 97, 'ti...",slider,6.660311,11,2,-7.474058,0,11,-36.446336,0,11,4.339689,100.0,100.0,slider_number,2
102,"{'z': 0.847412, 'duration': 231, 'activity': '...",slider,63.674192,61,3,-7.474058,0,61,-36.446336,0,61,2.674192,100.0,100.0,slider_number,2
103,"{'stability': 'In motion', 'z': 0.847412, 'tim...",slider,97.059559,94,4,-7.474058,0,94,-36.446336,0,94,3.059559,100.0,100.0,slider_number,2
104,"{'z': 0.847412, 'x': -0.25531, 'w': -0.189819,...",slider,89.814848,79,5,-7.474058,0,79,-36.446336,0,79,10.814848,100.0,100.0,slider_number,2
105,"{'calibration_status': 2, 'w': -0.189819, 'sta...",slider,15.061070,40,6,-7.474058,0,40,-36.446336,0,40,24.938930,100.0,100.0,slider_number,2
106,"{'activity_confidence': 97, 'x': -0.25531, 'du...",slider,23.348454,34,7,-7.474058,0,34,-36.446336,0,34,10.651546,100.0,100.0,slider_number,2
107,"{'x': -0.25531, 'duration': 231, 'y': -0.42511...",slider,50.490977,49,8,-7.474058,0,49,-36.446336,0,49,1.490977,100.0,100.0,slider_number,2
108,"{'calibration_status': 2, 'timestamp': 1719320...",slider,11.629489,16,9,-7.474058,0,16,-36.446336,0,16,4.370511,100.0,100.0,slider_number,2
109,"{'duration': 231, 'x': -0.25531, 'y': -0.42511...",slider,35.755673,39,10,-7.474058,0,39,-36.446336,0,39,3.244327,100.0,100.0,slider_number,2


In [35]:
df[df["experiment_type"] == "slider_greyscale"]

,sensorReading,inputType,value,truth,id,pitch,pitch_truth,pitch_truth_diff,roll,roll_truth,roll_truth_diff,slider_diff,pitch_truth_diff_percentage,roll_truth_diff_percentage,experiment_type,calibration
80,"{'activity': 'Still', 'stability': 'In motion'...",slider,5.326400,21,1,-7.474058,0,21,-36.446336,0,21,15.673600,100.0,100.0,slider_greyscale,2
81,"{'duration': 231, 'z': 0.847412, 'calibration_...",slider,12.359626,40,2,-7.474058,0,40,-36.446336,0,40,27.640374,100.0,100.0,slider_greyscale,2
82,"{'z': 0.847412, 'y': -0.42511, 'x': -0.25531, ...",slider,39.455783,39,3,-7.474058,0,39,-36.446336,0,39,0.455783,100.0,100.0,slider_greyscale,2
83,"{'y': -0.42511, 'calibration_status': 2, 'z': ...",slider,40.328140,16,4,-7.474058,0,16,-36.446336,0,16,24.328140,100.0,100.0,slider_greyscale,2
84,"{'stability': 'In motion', 'w': -0.189819, 'x'...",slider,22.779536,7,5,-7.474058,0,7,-36.446336,0,7,15.779536,100.0,100.0,slider_greyscale,2
85,"{'w': -0.189819, 'duration': 231, 'x': -0.2553...",slider,1.320304,2,6,-7.474058,0,2,-36.446336,0,2,0.679696,100.0,100.0,slider_greyscale,2
86,"{'y': -0.42511, 'w': -0.189819, 'activity': 'S...",slider,0.426171,1,7,-7.474058,0,1,-36.446336,0,1,0.573829,100.0,100.0,slider_greyscale,2
87,"{'calibration_status': 2, 'duration': 231, 'ti...",slider,0.000000,18,8,-7.474058,0,18,-36.446336,0,18,18.000000,100.0,100.0,slider_greyscale,2
88,"{'y': -0.42511, 'activity': 'Still', 'stabilit...",slider,25.844795,31,9,-7.474058,0,31,-36.446336,0,31,5.155205,100.0,100.0,slider_greyscale,2
89,"{'w': -0.189819, 'activity_confidence': 97, 'd...",slider,38.560388,12,10,-7.474058,0,12,-36.446336,0,12,26.560388,100.0,100.0,slider_greyscale,2


In [36]:
calibration_threshold = 2
g_pitch_greyscale_df = df[(df["experiment_type"] == "gesture_pitch_greyscale") & (df["calibration"] >= calibration_threshold)]
g_pitch_number_df = df[(df["experiment_type"] == "gesture_pitch_number") & (df["calibration"] >= calibration_threshold)]
g_roll_greyscale_df = df[(df["experiment_type"] == "gesture_roll_greyscale") & (df["calibration"] >= calibration_threshold)]
g_roll_number_df = df[(df["experiment_type"] == "gesture_roll_number") & (df["calibration"] >= calibration_threshold)]
g_slider_greyscale_df = df[(df["experiment_type"] == "slider_greyscale")]
g_slider_number_df = df[(df["experiment_type"] == "slider_number")]

fig = sp.make_subplots(rows=6, cols=2, subplot_titles=(
    'Pitch - Greyscale (Scaled 0-49)',
    'Pitch Error - Greyscale (Scaled 0-49)',
    'Pitch - Number (Scaled 0-100)',
    'Pitch Error - Number (Scaled 0-100)',
    'Roll - Greyscale (Scaled 0-49)',
    'Roll Error - Greyscale (Scaled 0-49)',
    'Roll - Number (Scaled 0-100)',
    'Roll Error - Number (Scaled 0-100)',
    'Slider - Greyscale (Scaled 0-49)',
    'Slider Error - Greyscale (Scaled 0-49)',
    'Slider - Number (Scaled 0-100)',
    'Slider Error - Number (Scaled 0-100)'
))

fig.add_trace(px.scatter(g_pitch_greyscale_df, x="truth", y="pitch_truth", color_discrete_sequence=['blue']).data[0], row=1, col=1)
fig.add_trace(px.scatter(g_pitch_greyscale_df, x="truth", y="pitch_truth", error_y="pitch_truth_diff", color_discrete_sequence=['blue']).data[0], row=1, col=2)
fig.add_trace(px.scatter(g_pitch_number_df, x="truth", y="pitch_truth", color_discrete_sequence=['green']).data[0], row=2, col=1)
fig.add_trace(px.scatter(g_pitch_number_df, x="truth", y="pitch_truth", error_y="pitch_truth_diff", color_discrete_sequence=['green']).data[0], row=2, col=2)
fig.add_trace(px.scatter(g_roll_greyscale_df, x="truth", y="roll_truth", color_discrete_sequence=['red']).data[0], row=3, col=1)
fig.add_trace(px.scatter(g_roll_greyscale_df, x="truth", y="roll_truth", error_y="roll_truth_diff", color_discrete_sequence=['red']).data[0], row=3, col=2)
fig.add_trace(px.scatter(g_roll_number_df, x="truth", y="roll_truth", color_discrete_sequence=['purple']).data[0], row=4, col=1)
fig.add_trace(px.scatter(g_roll_number_df, x="truth", y="roll_truth", error_y="roll_truth_diff", color_discrete_sequence=['purple']).data[0], row=4, col=2)
fig.add_trace(px.scatter(g_slider_greyscale_df, x="truth", y="value", color_discrete_sequence=['magenta']).data[0], row=5, col=1)
fig.add_trace(px.scatter(g_slider_greyscale_df, x="truth", y="value", error_y="slider_diff", color_discrete_sequence=['magenta']).data[0], row=5, col=2)
fig.add_trace(px.scatter(g_slider_number_df, x="truth", y="value", color_discrete_sequence=['purple']).data[0], row=6, col=1)
fig.add_trace(px.scatter(g_slider_number_df, x="truth", y="value", error_y="slider_diff", color_discrete_sequence=['purple']).data[0], row=6, col=2)

fig.update_layout(height=2000, width=1600, title_text="Sensor Readings Scatter Plots")
fig.update_xaxes(title_text="Truth Values")
fig.update_yaxes(title_text="Sensor Readings")

fig['layout']['yaxis1'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis2'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis3'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis4'].update(title='Pitch Sensor Readings')
fig['layout']['yaxis5'].update(title='Roll Sensor Readings')
fig['layout']['yaxis6'].update(title='Roll Sensor Readings')
fig['layout']['yaxis7'].update(title='Roll Sensor Readings')
fig['layout']['yaxis8'].update(title='Roll Sensor Readings')
fig['layout']['yaxis9'].update(title='Slider Readings')
fig['layout']['yaxis10'].update(title='Slider Readings')
fig['layout']['yaxis11'].update(title='Slider Readings')
fig['layout']['yaxis12'].update(title='Slider Readings')

fig.show()

In [40]:
def create_histogram_truth_plots(experiment_type, truth_type):
    filtered_df = df[(df['experiment_type'] == experiment_type) & 
                     (df['inputType'] == truth_type["input_type"])]

    fig = px.histogram(filtered_df, x=truth_type["truth_diff"],
                    title=f'{truth_type['label']} Differences - {experiment_type}',
                    labels={f'{truth_type["truth_diff"]}': f'{truth_type["label"]} Difference'})
    
    fig.show()
    
for experiment_type, truth_type in experiment_truth_type.items():
    create_histogram_truth_plots(experiment_type, truth_type)

In [37]:
def calculate_summary_statistics(experiment_type, truth_type):
    filtered_df = df[df['experiment_type'] == experiment_type]
    grouped = filtered_df.groupby('truth')[truth_type].agg(['mean', 'std', 'min', 'max', 'count', 'median'])
    grouped['iqr'] = filtered_df.groupby('truth')[truth_type].apply(iqr)
    grouped['skew'] = filtered_df.groupby('truth')[truth_type].apply(skew)
    grouped['kurtosis'] = filtered_df.groupby('truth')[truth_type].apply(kurtosis)
    
    # Kurtosis and skew calculate NaNs on certain truths (0 and 100), but it's not an issue. Filling NaNs with zeroes
    grouped = grouped.fillna(0)
    return grouped.reset_index()

def generate_summary_tables():
    summary_tables = {}
    for experiment_type, truth_info in experiment_truth_type.items():
        summary_table = calculate_summary_statistics(experiment_type, truth_info['truth'])
        summary_table['experiment_type'] = experiment_type
        summary_tables[experiment_type] = summary_table
    return summary_tables

def calculate_averages_per_experiment_type(summary_tables):
    averages = []
    for experiment_type, summary_table in summary_tables.items():
        averages.append({
            'experiment_type': experiment_type,
            'mean': summary_table['mean'].mean(),
            'std': summary_table['std'].mean(),
            'min': summary_table['min'].min(),
            'max': summary_table['max'].max(),
            'count': summary_table['count'].count(),
            'median': summary_table['median'].median(),
            'iqr': summary_table['iqr'].mean(),
            'skew': summary_table['skew'].mean(),
            'kurtosis': summary_table['kurtosis'].mean()
        })
    return pd.DataFrame(averages)

summary_tables = generate_summary_tables()
averages_df = calculate_averages_per_experiment_type(summary_tables)

display(averages_df)

for _, summary_table in summary_tables.items():
    display(summary_table)

,experiment_type,mean,std,min,max,count,median,iqr,skew,kurtosis
0,gesture_pitch_number,52.500000,0.000000,12.000000,79.000000,20,54.000000,0.000000,0.000000e+00,0.000000
1,gesture_pitch_greyscale,24.868421,1.228133,0.000000,49.000000,19,25.000000,0.868421,0.000000e+00,-0.105263
2,gesture_roll_number,66.000000,0.000000,37.000000,96.000000,20,70.500000,0.000000,0.000000e+00,0.000000
3,gesture_roll_greyscale,26.550000,0.000000,12.000000,43.000000,20,25.500000,0.000000,0.000000e+00,0.000000
4,slider_number,50.607768,0.000000,5.583582,97.059559,20,51.939619,0.000000,0.000000e+00,0.000000
5,slider_greyscale,20.057407,0.738555,0.000000,49.000000,19,12.359626,0.522237,1.531200e-17,-0.105263


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,4,12.0,0.0,12,12,1,12.0,0.0,0.0,0.0,gesture_pitch_number
1,17,27.0,0.0,27,27,1,27.0,0.0,0.0,0.0,gesture_pitch_number
2,28,30.0,0.0,30,30,1,30.0,0.0,0.0,0.0,gesture_pitch_number
3,35,38.0,0.0,38,38,1,38.0,0.0,0.0,0.0,gesture_pitch_number
4,36,39.0,0.0,39,39,1,39.0,0.0,0.0,0.0,gesture_pitch_number
5,44,46.0,0.0,46,46,1,46.0,0.0,0.0,0.0,gesture_pitch_number
6,50,54.0,0.0,54,54,1,54.0,0.0,0.0,0.0,gesture_pitch_number
7,53,55.0,0.0,55,55,1,55.0,0.0,0.0,0.0,gesture_pitch_number
8,56,50.0,0.0,50,50,1,50.0,0.0,0.0,0.0,gesture_pitch_number
9,57,52.0,0.0,52,52,1,52.0,0.0,0.0,0.0,gesture_pitch_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,0,32.5,23.334524,16,49,2,32.5,16.5,0.0,-2.0,gesture_pitch_greyscale
1,3,41.0,0.000000,41,41,1,41.0,0.0,0.0,0.0,gesture_pitch_greyscale
2,5,12.0,0.000000,12,12,1,12.0,0.0,0.0,0.0,gesture_pitch_greyscale
3,10,37.0,0.000000,37,37,1,37.0,0.0,0.0,0.0,gesture_pitch_greyscale
4,12,25.0,0.000000,25,25,1,25.0,0.0,0.0,0.0,gesture_pitch_greyscale
5,20,7.0,0.000000,7,7,1,7.0,0.0,0.0,0.0,gesture_pitch_greyscale
6,22,2.0,0.000000,2,2,1,2.0,0.0,0.0,0.0,gesture_pitch_greyscale
7,26,30.0,0.000000,30,30,1,30.0,0.0,0.0,0.0,gesture_pitch_greyscale
8,28,0.0,0.000000,0,0,1,0.0,0.0,0.0,0.0,gesture_pitch_greyscale
9,29,45.0,0.000000,45,45,1,45.0,0.0,0.0,0.0,gesture_pitch_greyscale


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,9,41.0,0.0,41,41,1,41.0,0.0,0.0,0.0,gesture_roll_number
1,20,37.0,0.0,37,37,1,37.0,0.0,0.0,0.0,gesture_roll_number
2,23,40.0,0.0,40,40,1,40.0,0.0,0.0,0.0,gesture_roll_number
3,26,40.0,0.0,40,40,1,40.0,0.0,0.0,0.0,gesture_roll_number
4,33,48.0,0.0,48,48,1,48.0,0.0,0.0,0.0,gesture_roll_number
5,37,54.0,0.0,54,54,1,54.0,0.0,0.0,0.0,gesture_roll_number
6,46,66.0,0.0,66,66,1,66.0,0.0,0.0,0.0,gesture_roll_number
7,54,69.0,0.0,69,69,1,69.0,0.0,0.0,0.0,gesture_roll_number
8,58,81.0,0.0,81,81,1,81.0,0.0,0.0,0.0,gesture_roll_number
9,60,77.0,0.0,77,77,1,77.0,0.0,0.0,0.0,gesture_roll_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,4,43.0,0.0,43,43,1,43.0,0.0,0.0,0.0,gesture_roll_greyscale
1,6,14.0,0.0,14,14,1,14.0,0.0,0.0,0.0,gesture_roll_greyscale
2,8,35.0,0.0,35,35,1,35.0,0.0,0.0,0.0,gesture_roll_greyscale
3,9,28.0,0.0,28,28,1,28.0,0.0,0.0,0.0,gesture_roll_greyscale
4,13,12.0,0.0,12,12,1,12.0,0.0,0.0,0.0,gesture_roll_greyscale
5,15,23.0,0.0,23,23,1,23.0,0.0,0.0,0.0,gesture_roll_greyscale
6,17,29.0,0.0,29,29,1,29.0,0.0,0.0,0.0,gesture_roll_greyscale
7,19,29.0,0.0,29,29,1,29.0,0.0,0.0,0.0,gesture_roll_greyscale
8,23,41.0,0.0,41,41,1,41.0,0.0,0.0,0.0,gesture_roll_greyscale
9,24,18.0,0.0,18,18,1,18.0,0.0,0.0,0.0,gesture_roll_greyscale


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,5,5.583582,0.0,5.583582,5.583582,1,5.583582,0.0,0.0,0.0,slider_number
1,10,9.044641,0.0,9.044641,9.044641,1,9.044641,0.0,0.0,0.0,slider_number
2,11,6.660311,0.0,6.660311,6.660311,1,6.660311,0.0,0.0,0.0,slider_number
3,16,11.629489,0.0,11.629489,11.629489,1,11.629489,0.0,0.0,0.0,slider_number
4,27,26.547758,0.0,26.547758,26.547758,1,26.547758,0.0,0.0,0.0,slider_number
5,31,22.167535,0.0,22.167535,22.167535,1,22.167535,0.0,0.0,0.0,slider_number
6,34,23.348454,0.0,23.348454,23.348454,1,23.348454,0.0,0.0,0.0,slider_number
7,39,35.755673,0.0,35.755673,35.755673,1,35.755673,0.0,0.0,0.0,slider_number
8,40,15.061070,0.0,15.061070,15.061070,1,15.061070,0.0,0.0,0.0,slider_number
9,49,50.490977,0.0,50.490977,50.490977,1,50.490977,0.0,0.0,0.0,slider_number


,truth,mean,std,min,max,count,median,iqr,skew,kurtosis,experiment_type
0,1,0.426171,0.000000,0.426171,0.426171,1,0.426171,0.00000,0.000000e+00,0.0,slider_greyscale
1,2,11.242814,14.032548,1.320304,21.165323,2,11.242814,9.92251,2.909281e-16,-2.0,slider_greyscale
2,6,30.175503,0.000000,30.175503,30.175503,1,30.175503,0.00000,0.000000e+00,0.0,slider_greyscale
3,7,22.779536,0.000000,22.779536,22.779536,1,22.779536,0.00000,0.000000e+00,0.0,slider_greyscale
4,9,49.000000,0.000000,49.000000,49.000000,1,49.000000,0.00000,0.000000e+00,0.0,slider_greyscale
5,12,38.560388,0.000000,38.560388,38.560388,1,38.560388,0.00000,0.000000e+00,0.0,slider_greyscale
6,13,2.565144,0.000000,2.565144,2.565144,1,2.565144,0.00000,0.000000e+00,0.0,slider_greyscale
7,15,7.510580,0.000000,7.510580,7.510580,1,7.510580,0.00000,0.000000e+00,0.0,slider_greyscale
8,16,40.328140,0.000000,40.328140,40.328140,1,40.328140,0.00000,0.000000e+00,0.0,slider_greyscale
9,18,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.00000,0.000000e+00,0.0,slider_greyscale
